# Revised case normalization for Hirslanden Aarau 2018

Notes:

- 17 of 17 revised DtoD cases were joined with the BFS cases from the DB.



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.py.global_configs import *
from src.revised_case_normalization.py.normalize import normalize
from src.revised_case_normalization.py.revise import revise
from src.revised_case_normalization.py.group import group
from src.revised_case_normalization.py.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
                     'Hirslanden Aarau', '2018', 'Aarau 2018')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI_Aarau_Birshof_ST. Anna.xlsx', hospital_name_db='Hirslanden Aarau', year='2018', sheet='Aarau 2018')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-03 22:45:02.825 | INFO     | src.revised_case_normalization.py.normalize:normalize:35 - Read 17 cases for Hirslanden Aarau 2018
2022-11-03 22:45:02.831 | INFO     | src.revised_case_normalization.py.normalize:normalize:59 - TYPES:
tranche                   string
datum                     string
case_id                   object
patient_id                object
kkik                      string
gender                    string
age_years                  int64
bfs_code                  string
duration_of_stay           int64
pflegetage neu            string
pccl                       int64
pccl neu                  string
old_pd                    string
primary_diagnosis         string
added_icds                string
removed_icds              string
added_chops               string
removed_chops             string
drg                       string
drg neu                   string
cw alt                    string
cw neu                    string
cw-änderung möglich       string
cw

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,0041623461,0001059029,W,77,2,41623461,I7024,I7024,M100,3,F59E,[J4481],[J4483],[],[]
1,0041665678,0002043896,M,82,17,41665678,I5001,I5001,M100,3,F62C,[R18],[],[],[]
2,0041698586,0022177655,M,93,2,41698586,I7024,I7024,M100,3,F59E,[I743],[],[],[]
3,0041719219,0001339311,M,91,4,41719219,I495,I495,M100,3,F71B,[N182],[],[],[]
4,41777951,0022177655,M,93,2,41777951,I7024,I7024,M100,0,F59E,"[N184, T828]",[],[],[]
5,0041648887,0001709901,M,64,8,41648887,I5014,I5014,M100,3,F62C,[E1191],[E1190],[],[]
6,0041709615,0001749027,W,58,5,41709615,N390,N390,M100,3,L63D,[B370],[],[],[]
7,0041772407,0022190089,W,79,9,41772407,N390,N390,M100,3,L63D,[E440],[],[890A32],[]
8,0041769953,0022179314,M,70,6,41769953,N1781,N1781,M100,3,L60D,[J9610],[],[],[]
9,0041803251,0001720953,M,51,8,41803251,I208,I208,M200,3,F06D,[J9580],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases

2022-11-03 22:45:05.185 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:125 - Read 10112 rows from the DB, for the hospital 'Hirslanden Aarau' in 2018


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,115875,0041623461,I7024,"[I7020, Z9588, I1090, N184, N390, ...]",395014:R:20180111,"[395011:R:20180111, 397510:R:20180111, 004C12::20180111, 005599:R:20180111, 0043:R:20180111, ...]",W,77,0,0,2,0,01,2018-01-10,0,00,2018-01-12
1,116472,0041648887,I5014,"[I5001, I2513, I2522, N184, I1320, ...]",99B712::20180117,"[8945::20180117, 860A06:L:20180122, 948X40::20180122]",M,64,0,0,8,8,01,2018-01-17,0,00,2018-01-25
2,116851,0041665678,I5001,"[J91, I5013, I7020, L304, N185, ...]",887211::20180130,"[887963:B:20180130, 887964:B:20180130, 887975:B:20180130, 3491:R:20180201]",M,82,0,0,17,0,01,2018-01-30,0,00,2018-02-16
3,117722,0041698586,I7024,"[Z9588, N184, Z922, I743]",395011:R:20180315,"[397511:R:20180315, 395021:R:20180315, 397510:R:20180315, 0042::20180315, 004B18:R:20180315, ...]",M,93,0,0,2,0,01,2018-03-15,0,00,2018-03-17
4,117999,0041709615,N390,"[B965, R400, G3520, F446, Z936, ...]",992217::20180309,[],W,58,0,0,5,0,01,2018-03-09,0,00,2018-03-14
5,118215,0041719219,I495,"[I484, I480, I1090, I2519, I340, ...]",893909::20180317,[],M,91,0,0,4,0,01,2018-03-17,0,00,2018-03-21
6,119991,0041769953,N1781,"[Y579, I1100, I480, I5001, G2010, ...]",992909::20180519,[],M,70,0,0,6,0,01,2018-05-19,0,00,2018-05-25
7,120078,0041772407,N390,"[B962, A4158, R650, E1190, E788, ...]",992217::20180524,[890A32::],W,79,0,0,9,0,01,2018-05-23,0,00,2018-06-01
8,120279,41777951,I7024,"[Z9588, Z921, Z922, N184, T828]",395014:R:20180601,"[004C12::20180601, 395011:R:20180601, 0043::20180601, 004B1A:R:20180601, 004B46:R:20180601, ...]",M,93,0,0,2,0,01,2018-05-31,0,00,2018-06-02
9,121288,0041803251,I208,"[I2513, D684, U6912, E788, J9580]",361226::20180705,"[361121::20180705, 361122::20180705, 361C14::20180705, 3963::20180705, 3964::20180705, ...]",M,51,0,0,8,8,01,2018-07-04,0,00,2018-07-12


In [5]:
if unmatched.shape[0] > 0:
    unmatched

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)

2022-11-03 22:45:06.040 | INFO     | src.revised_case_normalization.py.group:group:12 - Grouping 17 cases ...
2022-11-03 22:45:07.626 | SUCCESS  | src.revised_case_normalization.py.group:group:16 - Grouped 17 cases into: 17 revisions, 168 diagnoses rows, 108 procedure rows


In [7]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-03 22:45:07.635 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:339 - Trying to insert 17 cases into the 'Revisions' table ...
2022-11-03 22:45:08.232 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:365 - Deleted 204 rows from the "Revisions" table, which is about to be updated
2022-11-03 22:45:08.329 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:376 - Inserted 17 cases into the 'Revisions' table
2022-11-03 22:45:08.330 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:388 - Trying to insert 168 rows into the 'Diagnoses' table ...
2022-11-03 22:45:09.998 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:426 - Inserted 168 rows into the 'Diagnoses' table
2022-11-03 22:45:09.999 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:436 - Trying to insert 108 rows into the 'Procedures' t